# Initial CNN Implementation [DRAFT]
Draft of the initial implementation of the CNN model for the Group's project. Created with the objective of understanding the basic structure of the model and its implementation in Python.

# Initial Setup

## Install required packages

In [ ]:
!pip install tensorflow
!pip install keras
!pip install numpy
!pip install pandas
!pip install matplotlib
!pip install opencv-python

## Import libraries

In [ ]:
import os
import cv2
import keras
import random
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
from tensorflow import data as tf_data
from tensorflow import image as tf_image
from google.colab.patches import cv2_imshow

# Auxiliary Functions and Methods

In [ ]:
# Add here all auxiliary functions (dataset split methods, formatting and conversion methods...)

# Section 3: Image Processing Pipeline

## Processing Pipeline Class

The `ProcessingPipeline` class orchestrates the application of 

*   Item da lista
*   Item da lista

filters and augmentations to the image data. It automates the process of image enhancement and prepares the data for segmentation.

In [ ]:
class ProcessingPipeline:
    """
    Manages the application of image processing filters and augmentations.

    Attributes:
        filters (list): A list of filter objects to apply to the images.
        augmentations (list): A list of augmentation objects to apply to the images.
        history (list): Records outcomes of applied filters and augmentations for visualization.
    """

    def __init__(self, plot_storyline=False):
        """
        Initializes the processing pipeline with empty lists for filters, augmentations, and history.
        """
        self.filters = []
        self.augmentations = []
        self.history = []

    def add_filters(self, filters):
        """
        Adds multiple filter objects to the pipeline.

        Parameters:
            filters (list): List of filter objects to be added.
        """
        self.filters.extend(filters)

    def clear_filters(self):
        """
        Clears all filter objects from the pipeline.
        """
        self.filters = []

    def add_augmentations(self, augmentations):
        """
        Adds multiple augmentation objects to the pipeline.

        Parameters:
            augmentations (list): List of augmentation objects to be added.
        """
        self.augmentations.extend(augmentations)

    def clear_augmentations(self):
        """
        Clears all augmentation objects from the pipeline.
        """
        self.augmentations = []

    def apply_filters(self, img):
        """
        Applies each filter in sequence to the image.

        Parameters:
            img (numpy.ndarray): The original image to be processed.

        Returns:
            numpy.ndarray: The image processed by all filters.
        """
        _img = img
        for _filter in self.filters:
            _img, _ = _filter.apply(_img, None)
            self.history.append((_img.copy(), type(_filter).__name__, "Filter"))
        return _img

    def apply_crop(self, img, mask, new_width=120, new_height=120, n=3):
        """
        Randomly crops the given image and mask arrays into 'n' new images and masks with dimensions 'new_width' x 'new_height'.

        Parameters:
            img (numpy.ndarray): The numpy array representing the original image.
            mask (numpy.ndarray): The numpy array representing the original mask.
            n (int): The number of new images and masks to generate.
            new_width (int): The width of the new images and masks.
            new_height (int): The height of the new images and masks.

        Returns:
            tuple: A tuple containing three elements:
                   - A list of numpy.ndarray representing the cropped images.
                   - A list of numpy.ndarray representing the cropped masks.
                   - A list of tuples containing the top-left corner coordinates of each cropped area.
        """
        original_height, original_width = img.shape[:2]
        if new_width > original_width or new_height > original_height:
            print(original_width)
            raise ValueError(
                "New dimensions must be smaller than the original dimensions."
            )

        cropped_images, cropped_masks, crop_coordinates = [], [], []
        for _ in range(n):
            top = np.random.randint(0, original_height - new_height + 1)
            left = np.random.randint(0, original_width - new_width + 1)
            crop_slice = np.s_[top : top + new_height, left : left + new_width]
            cropped_images.append(img[crop_slice])
            cropped_masks.append(mask[crop_slice])
            crop_coordinates.append((left, top))
            self.history.append(
                (img[crop_slice], f"Cropped Image at ({left}, {top})", "Crop")
            )

        return cropped_images, cropped_masks, crop_coordinates

    def apply_augmentations(self, images, masks, n=3):
        """
        Applies data augmentation to a list of images and their corresponding masks.

        Parameters:
            images (list of numpy.ndarray): The list of numpy arrays representing the original images.
            masks (list of numpy.ndarray): The list of numpy arrays representing the masks for the images.
            n (int): Number of augmentations to apply to each image.
            filters (list): List of instantiated filter classes to apply.

        Returns:
            tuple: A tuple containing two elements:
                   - List of numpy.ndarray representing the original and augmented images.
                   - List of numpy.ndarray representing the original and augmented masks.
        """
        all_images = []
        all_masks = []

        for image, mask in zip(images, masks):
            augmented_images = [image]
            augmented_masks = [mask]
            previous_transformations = set()

            while len(augmented_images) - 1 < n:
                selected_filter = random.choice(self.augmentations)
                transformation_key = (
                    type(selected_filter).__name__,
                    tuple(selected_filter.__dict__.values()),
                )

                if transformation_key not in previous_transformations:
                    augmented_image, augmented_mask = selected_filter.apply(image, mask)
                    augmented_images.append(augmented_image)
                    augmented_masks.append(augmented_mask)
                    previous_transformations.add(transformation_key)
                    self.history.append(
                        (
                            augmented_image,
                            f"Augmented with {type(selected_filter).__name__}",
                            "Augmentation",
                        )
                    )
                    self.history.append(
                        (
                            augmented_mask,
                            f"[MASK] Augmented with {type(selected_filter).__name__}",
                            "Augmentation",
                        )
                    )

            all_images.extend(augmented_images)
            all_masks.extend(augmented_masks)

        return all_images, all_masks

    def apply_normalization(self, imgs, masks):
        """
        Normalizes the pixel values of images and masks to the range [0, 1].

        Parameters:
            imgs (list of numpy.ndarray): The list of images to be normalized.
            masks (list of numpy.ndarray): The list of masks to be normalized.

        Returns:
            tuple: A tuple containing two elements:
                   - List of numpy.ndarray representing the normalized images.
                   - List of numpy.ndarray representing the normalized masks.
        """
        _imgs = []
        _masks = []
        for index, _img in enumerate(imgs):
            height, width, _ = _img.shape
            m_height, m_width, _ = masks[index].shape
            norm_img = np.zeros((height, width))
            norm_mask = np.zeros((m_height, m_width))
            norm_img = cv.normalize(_img, norm_img, 0, 255, cv.NORM_MINMAX)
            norm_mask = cv.normalize(masks[index], norm_mask, 0, 255, cv.NORM_MINMAX)
            norm_img = norm_img / 255
            norm_mask = norm_mask / 255
            _imgs.append(norm_img)
            _masks.append(norm_mask)
        return _imgs, _masks

    def run(self, img, mask, n_augmented, crop_size=120, n_crop=20):
        """
        Executes the entire image processing pipeline.

        Parameters:
            img (numpy.ndarray): The original image to process.
            mask (numpy.ndarray): The associated mask for the image.
            n_augmented (int): The number of augmented images to generate.
            crop_size (int): The size for cropping the images.
            n_crop (int): The number of crops to produce.

        Returns:
            tuple: A tuple containing three elements:
                   - List of numpy.ndarray representing the normalized images.
                   - List of numpy.ndarray representing the normalized masks.
                   - List of tuples containing the coordinates of cropped areas.
        """
        highlighted_img = self.apply_filters(img)
        cropped_imgs, cropped_masks, cropped_coordinates = self.apply_crop(
            highlighted_img, mask, new_height=crop_size, new_width=crop_size, n=n_crop
        )
        augmented_imgs, augmented_masks = self.apply_augmentations(
            cropped_imgs, cropped_masks, n_augmented
        )
        normalized_imgs, normalized_masks = self.apply_normalization(
            augmented_imgs, augmented_masks
        )
        return normalized_imgs, normalized_masks, cropped_coordinates

    def get_history(self):
        return self.history


## Image Processors

Here we define a suite of image processing classes that extend `BaseImageProcess`. Each class implements a specific image processing technique, such as rotation, blurring, and thresholding, which are essential for the feature extraction phase of plot segmentation.



In [ ]:
class BaseImageProcess:
    """
    BaseImageProcess: A base class for image processing algorithms.

    This class provides a basic framework for implementing image processing algorithms and is intended to be subclassed.
    Subclasses should implement the `apply` method to perform specific image processing operations on an input image.
    """

    def apply(self, img, mask=None):
        """
        Placeholder for applying an image processing algorithm.

        Args:
            img: The input image to process.

        Returns:
            The processed image.
        """
        pass


class Rotate(BaseImageProcess):
    def __init__(self):
        self.angle = random.choice([-15, -10, -5, 5, 10, 15])

    def apply(self, img, mask=None):
        height, width = img.shape[:2]
        rotation_matrix = cv.getRotationMatrix2D((width / 2, height / 2), self.angle, 1)
        return cv.warpAffine(img, rotation_matrix, (width, height)), (
            cv.warpAffine(mask, rotation_matrix, (width, height))
            if mask is not None
            else None
        )


class BilateralFilter(BaseImageProcess):
    """
    BilateralFilter: Applies bilateral filtering to an image to reduce noise while keeping edges sharp.
    """

    def __init__(self, d=9, sigmaColor=75, sigmaSpace=75):
        self.d = d
        self.sigmaColor = sigmaColor
        self.sigmaSpace = sigmaSpace

    def apply(self, img, mask=None):
        return cv.bilateralFilter(img, self.d, self.sigmaColor, self.sigmaSpace), mask


class Translate(BaseImageProcess):
    """
    Applies translation to an image using random horizontal and vertical shifts.

    Attributes:
        dx (int): Horizontal shift, chosen randomly from a specified range.
        dy (int): Vertical shift, chosen randomly from a specified range.
    """

    def __init__(self):
        self.dx = random.choice([-10, -5, 0, 5, 10])
        self.dy = random.choice([-10, -5, 0, 5, 10])

    def apply(self, img, mask=None):
        translation_matrix = np.float32([[1, 0, self.dx], [0, 1, self.dy]])
        height, width = img.shape[:2]
        return cv.warpAffine(img, translation_matrix, (width, height)), (
            cv.warpAffine(mask, translation_matrix, (width, height))
            if mask is not None
            else None
        )


class Flip(BaseImageProcess):
    """
    Flips an image either horizontally, vertically, or both, based on a randomly selected flip type.

    Attributes:
        flip_type (int): Type of flip to apply; -1 for both axes, 0 for vertical, 1 for horizontal.
    """

    def __init__(self):
        self.flip_type = random.choice([-1, 0, 1])

    def apply(self, img, mask=None):
        return cv.flip(img, self.flip_type), (
            cv.flip(mask, self.flip_type) if mask is not None else None
        )


class BrightnessContrast(BaseImageProcess):
    """
    Adjusts the brightness and contrast of an image using random values.

    Attributes:
        alpha (float): Factor by which the contrast will be adjusted.
        beta (int): Value that will be added to the pixels for brightness adjustment.
    """

    def __init__(self):
        self.alpha = random.uniform(0.5, 1.5)
        self.beta = random.randint(-50, 50)

    def apply(self, img, mask=None):
        return cv.convertScaleAbs(img, alpha=self.alpha, beta=self.beta), mask


class MedianBlur(BaseImageProcess):
    """
    Applies median blurring to an image using a randomly chosen kernel size.

    Attributes:
        kernel_size (int): The size of the kernel used, selected randomly from a set of possible odd sizes.
    """

    def __init__(self):
        self.kernel_size = random.choice([3, 5, 7, 9, 11])

    def apply(self, img, mask=None):
        return cv.medianBlur(img, self.kernel_size), mask


class RandomGaussianBlur(BaseImageProcess):
    """
    Applies Gaussian blur filtering to an image with a randomly chosen kernel size.

    Attributes:
        kernel_size (int): Size of the Gaussian blur kernel, selected randomly.
    """

    def __init__(self):
        self.kernel_size = random.choice([3, 5, 7, 9, 11])

    def apply(self, img, mask=None):
        return cv.GaussianBlur(img, (self.kernel_size, self.kernel_size), 0), mask


class GaussianBlur(BaseImageProcess):
    """
    GaussianBlur: Applies Gaussian blur filtering to an image.

    This class provides an implementation of Gaussian blur filtering, commonly used to reduce image noise and detail.

    Attributes:
        kernel_size (int): Size of the kernel used for the Gaussian filter.
    """

    def __init__(self, kernel_size=5):
        self.kernel_size = kernel_size

    def apply(self, img, mask=None):
        return cv.GaussianBlur(img, (self.kernel_size, self.kernel_size), 0), mask


class BinaryThresh(BaseImageProcess):
    """
    BinaryThresh: Applies binary thresholding to an image.

    Binary thresholding converts an image to binary (black and white) based on a threshold value. Pixels above the
    threshold are set to the maximum value, and those below are set to zero.

    Attributes:
        thresh (int): Threshold value.
        max_val (int): Maximum value to use with the threshold.
    """

    def __init__(self, thresh=127, max_val=255):
        self.thresh = thresh
        self.max_val = max_val

    def apply(self, img, mask=None):
        _img = cv.cvtColor(img, cv.COLOR_BGR2GRAY) if len(img.shape) > 2 else img
        _, _img = cv.threshold(_img, self.thresh, self.max_val, cv.THRESH_BINARY)
        return _img, mask


class AdaptiveMeanThresh(BaseImageProcess):
    """
    AdaptiveMeanThresh: Applies adaptive mean thresholding to an image.

    Unlike simple thresholding, adaptive thresholding changes the threshold dynamically over the image based on local
    image characteristics.

    Attributes:
        block_size (int): Size of a pixel neighborhood used to calculate the threshold.
        c (int): Constant subtracted from the calculated mean or weighted mean.
    """

    def __init__(self, block_size=11, c=2):
        self.block_size = block_size
        self.c = c

    def apply(self, img, mask=None):
        _img = cv.cvtColor(img, cv.COLOR_BGR2GRAY) if len(img.shape) > 2 else img
        return (
            cv.adaptiveThreshold(
                _img,
                255,
                cv.ADAPTIVE_THRESH_MEAN_C,
                cv.THRESH_BINARY,
                self.block_size,
                self.c,
            ),
            mask,
        )


class AdaptiveGaussThresh(BaseImageProcess):
    """
    AdaptiveGaussThresh: Applies adaptive Gaussian thresholding to an image.

    This method uses a weighted sum of neighbourhood values where weights are a Gaussian window, which provides
    a more natural thresholding, especially under varying illumination.

    Attributes:
        block_size (int): Size of a pixel neighborhood used to calculate the threshold.
        c (int): Constant subtracted from the calculated weighted sum.
    """

    def __init__(self, block_size=11, c=2):
        self.block_size = block_size
        self.c = c

    def apply(self, img, mask=None):
        _img = cv.cvtColor(img, cv.COLOR_BGR2GRAY) if len(img.shape) > 2 else img
        return (
            cv.adaptiveThreshold(
                _img,
                255,
                cv.ADAPTIVE_THRESH_GAUSSIAN_C,
                cv.THRESH_BINARY,
                self.block_size,
                self.c,
            ),
            mask,
        )


class OtsuThresh(BaseImageProcess):
    """
    OtsuThresh: Applies Otsu's thresholding to automatically perform histogram shape-based image thresholding.

    This method is useful when the image contains two prominent pixel intensities and calculates an optimal threshold
    separating these two classes so that their combined spread (intra-class variance) is minimal.
    """

    def apply(self, img, mask=None):
        _img = cv.cvtColor(img, cv.COLOR_BGR2GRAY) if len(img.shape) > 2 else img
        _, _img = cv.threshold(_img, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)
        return _img, mask


class MorphDilate(BaseImageProcess):
    """
    MorphDilate: Applies morphological dilation to an image.

    Dilation increases the white region in the image or size of the foreground object. Commonly used to accentuate
    features.

    Attributes:
        kernel_size (int): Size of the structuring element.
        iterations (int): Number of times dilation is applied.
    """

    def __init__(self, kernel_size=3, iterations=2):
        self.kernel_size = kernel_size
        self.iterations = iterations
        self.kernel = np.ones((self.kernel_size, self.kernel_size), np.uint8)

    def apply(self, img, mask=None):
        return cv.dilate(img, self.kernel, iterations=self.iterations), mask


class MorphErode(BaseImageProcess):
    """
    MorphErode: Applies morphological erosion to an image.

    Erosion erodes away the boundaries of the foreground object and is used to diminish the features of an image.

    Attributes:
        kernel_size (int): Size of the structuring element.
        iterations (int): Number of times erosion is applied.
    """

    def __init__(self, kernel_size=3, iterations=2):
        self.kernel_size = kernel_size
        self.iterations = iterations
        self.kernel = np.ones((self.kernel_size, self.kernel_size), np.uint8)

    def apply(self, img, mask=None):
        return cv.erode(img, self.kernel, iterations=self.iterations), mask


class LoG(BaseImageProcess):
    """
    LoG: Applies Laplacian of Gaussian filtering to an image.

    This method is used to highlight regions of rapid intensity change and is therefore often used for edge detection.
    First, it applies a Gaussian blur, then computes the Laplacian of the result.

    Attributes:
        sigma (float): Standard deviation of the Gaussian filter.
        size (int): Size of the filter kernel.
    """

    def __init__(self, sigma=2.0, size=None):
        self.sigma = sigma
        self.size = (
            size
            if size is not None
            else int(6 * self.sigma + 1) if self.sigma >= 1 else 7
        )
        if self.size % 2 == 0:
            self.size += 1

    def apply(self, img, mask=None):
        x, y = np.meshgrid(
            np.arange(-self.size // 2 + 1, self.size // 2 + 1),
            np.arange(-self.size // 2 + 1, self.size // 2 + 1),
        )
        kernel = (
            -(1 / (np.pi * self.sigma**4))
            * (1 - ((x**2 + y**2) / (2 * self.sigma**2)))
            * np.exp(-(x**2 + y**2) / (2 * self.sigma**2))
        )
        kernel = kernel / np.sum(np.abs(kernel))
        return cv.filter2D(img, -1, kernel), mask


class LoGConv(BaseImageProcess):
    """
    LoGConv: Implements convolution with a Laplacian of Gaussian kernel to an image.

    Similar to the LoG class, but tailored for applying custom convolution operations directly with a manually
    crafted LoG kernel.

    Attributes:
        sigma (float): Standard deviation of the Gaussian filter.
        size (int): Size of the filter kernel.
    """

    def __init__(self, sigma=2.0, size=None):
        self.sigma = sigma
        self.size = size if size is not None else int(6 * sigma + 1)
        if self.size % 2 == 0:
            self.size += 1

    def apply(self, img, mask=None):
        if len(img.shape) == 3:
            img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        x, y = np.meshgrid(
            np.arange(-self.size // 2 + 1, self.size // 2 + 1),
            np.arange(-self.size // 2 + 1, self.size // 2 + 1),
        )
        kernel = (
            -(1 / (np.pi * self.sigma**4))
            * (1 - ((x**2 + y**2) / (2 * self.sigma**2)))
            * np.exp(-(x**2 + y**2) / (2 * self.sigma**2))
        )
        kernel = kernel / np.sum(np.abs(kernel))
        if len(img.shape) == 3:
            img = cv.cvtColor(img, cv.COLOR_GRAY2BGR)
            img = convolve(img, kernel)
            img = np.clip(img, 0, 255).astype(np.uint8)
        else:
            img = convolve(img, kernel)
        return img, mask

# Section 4: Auxiliary Functions

## Helper Functions

Auxiliary functions such as `load_image` and `display_history` are defined here. These functions support the main pipeline by providing image loading capabilities and visualizing the processing history.

In [ ]:
def load_image(path, color_mode=cv.IMREAD_COLOR):
    """Loads an image from the given path with specified color mode."""
    image = cv.imread(path, color_mode)
    if image is None:
        raise FileNotFoundError(f"Could not load image from {path}")
    return image

def display_history(history):
    n_images = len(history)
    n_rows = math.ceil(n_images ** 0.5)
    n_cols = math.ceil(n_images / n_rows)
    plt.figure(figsize=(n_cols * 4, n_rows * 4))
    for i, (img, label, _) in enumerate(history):
        plt.subplot(n_rows, n_cols, i + 1)
        plt.imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))
        plt.title(label)
        plt.axis('off')
    plt.tight_layout()


def get_dataset(batch_size,
    img_size,
    input_img_arr,
    target_img_arr,
    max_dataset_len=None,
):
    """Returns a TF Dataset."""
    if max_dataset_len:
        input_img_arr = input_img_arr[:max_dataset_len]
        target_img_arr = target_img_arr[:max_dataset_len]
    dataset = tf_data.Dataset.from_tensor_slices((input_img_arr, target_img_arr))
    return dataset.batch(batch_size)

# Section 5: Execution of Processing Pipeline

## Pipeline Configuration

Set up the processing pipeline by selecting the desired filters and augmentations. This configuration will determine how the images are processed and enhanced.

In [ ]:
n_augmented = 5
filters = []
augmentations = []
augmentations = [Rotate(), Translate(), Flip(), BrightnessContrast(), RandomGaussianBlur(), MedianBlur()]

pipeline = ProcessingPipeline()
pipeline.add_filters(filters)
pipeline.add_augmentations(augmentations)

base_masks_path = "/content/drive/MyDrive/Images/Masks"
base_inputs_path = "/content/drive/MyDrive/Images/Input"
image_data_manager = ImageDataManager(base_masks_path, base_inputs_path)

images_np_array = []
masks_np_array = []

for key in image_data_manager.objects.keys():
  imgs = image_data_manager.objects[key]['images']
  imgs_filtered = []

  for img in imgs:
    if img.shape[1] == 1200:
      imgs_filtered.append(np.transpose(img, (1, 2, 0)))

  image_data_manager.objects[key]['images'] = imgs_filtered
  mask = image_data_manager.objects[key]['mask']
  image_data_manager.objects[key]['mask'] = mask
  img = imgs_filtered[-1]

  n_crop = img.shape[0] // 128
  imgs, masks, coordinates = pipeline.run(img, mask, crop_size=128, n_crop=n_crop, n_augmented=5)
  images_np_array.extend(imgs)
  masks_np_array.extend(masks)
  

count = len(images_np_array)
print(f"Total Image count: {count} images")

### History View
Views the history of the processing pipeline for a given image.

In [ ]:
display_history(pipeline.history[:100])

# Section 6 - Dataset

## Split Dataset to Train / Test

In [ ]:
val_samples = int(len(images_np_array) * 0.25)
random.Random(800).shuffle(images_np_array)
random.Random(800).shuffle(masks_np_array)
train_input_img_sample = images_np_array[:-val_samples]
train_target_img_sample = masks_np_array[:-val_samples]
val_input_img_sample = images_np_array[-val_samples:]
val_target_img_sample = masks_np_array[-val_samples:]

img_size = (128,128)
batch_size = 4

# training dataset
train_dataset = get_dataset(
    batch_size,
    img_size,
    images_np_array,
    masks_np_array,
    max_dataset_len=2000,
)

# validation dataset
valid_dataset = get_dataset(
    batch_size, img_size, val_input_img_sample, val_target_img_sample
)

# CNN Model

## Model Hyperparameters

In [ ]:
epochs = 50

## Model Architecture

In [ ]:
def xception_model(img_size):
    inputs = Input(shape=img_size + (3,))

    x = Conv2D(32, 3, strides=2, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    previous_block_activation = x

    for filters in [64, 128, 256]:
        x = Activation("relu")(x)
        x = SeparableConv2D(filters, 3, padding="same")(x)
        x = BatchNormalization()(x)

        x = Activation("relu")(x)
        x = SeparableConv2D(filters, 3, padding="same")(x)
        x = BatchNormalization()(x)

        x = MaxPooling2D(3, strides=2, padding="same")(x)

        residual = Conv2D(filters, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = add([x, residual]) 
        previous_block_activation = x

    for filters in [256, 128, 64, 32]:
        x = Activation("relu")(x)
        x = Conv2DTranspose(filters, 3, padding="same")(x)
        x = BatchNormalization()(x)

        x = Activation("relu")(x)
        x = Conv2DTranspose(filters, 3, padding="same")(x)
        x = BatchNormalization()(x)

        x = UpSampling2D(2)(x)

        residual = UpSampling2D(2)(previous_block_activation)
        residual = Conv2D(filters, 1, padding="same")(residual)
        x = add([x, residual])
        previous_block_activation = x

    outputs = Conv2D(1, 3, activation="sigmoid",   padding="same")(x)
    model = Model(inputs, outputs)
    return model

## Loss Functions

### Class Definition

In [ ]:
import tensorflow as tf
from keras import backend as K

class LossFunctions:
    """
    A collection of loss functions and corresponding metrics for image segmentation tasks.
    """

    def binary_crossentropy():
        """
        Returns the binary cross-entropy loss function configured with from_logits=True.
        
        Returns:
            An instance of tf.keras.losses.BinaryCrossentropy configured with from_logits=True.
        """
        return tf.keras.losses.BinaryCrossentropy(from_logits=True)
    
    def binary_accuracy():
        """
        Returns the binary accuracy metric.

        Returns:
            An instance of tf.keras.metrics.BinaryAccuracy.
        """
        return tf.keras.metrics.BinaryAccuracy()
    
    def dice_coefficient(y_true, y_pred, smooth=1):
        """
        Computes the Dice Coefficient, a measure of overlap between true and predicted labels.
        
        Args:
            y_true: Ground truth labels.
            y_pred: Predicted labels.
            smooth: A smoothing constant to avoid division by zero.

        Returns:
            A tensor representing the Dice Coefficient.
        """
        y_true_f = K.flatten(K.cast(y_true, 'float32'))
        y_pred_f = K.flatten(y_pred)
        intersection = K.sum(y_true_f * y_pred_f)
        return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

    def dice_loss(y_true, y_pred):
        """
        Computes the Dice Loss, derived from the Dice Coefficient, used to maximize overlap.
        
        Args:
            y_true: Ground truth labels.
            y_pred: Predicted labels.

        Returns:
            A tensor representing the Dice Loss.
        """
        return 1 - LossFunctions.dice_coefficient(y_true, y_pred)
    
    def jaccard_index(y_true, y_pred, smooth=1):
        """
        Computes the Jaccard Index, also known as Intersection over Union (IoU), 
        which measures the overlap between true and predicted labels.
        
        Args:
            y_true: Ground truth labels.
            y_pred: Predicted labels.
            smooth: A smoothing constant to avoid division by zero.

        Returns:
            A tensor representing the Jaccard Index.
        """
        y_true_f = K.flatten(K.cast(y_true, 'float32'))
        y_pred_f = K.flatten(y_pred)
        intersection = K.sum(y_true_f * y_pred_f)
        sum_ = K.sum(y_true_f) + K.sum(y_pred_f)
        jac = (intersection + smooth) / (sum_ - intersection + smooth)
        return jac
    
    def jaccard_loss(y_true, y_pred):
        """
        Computes the Jaccard Loss, derived from the Jaccard Index, used to maximize 
        the intersection over union.

        Args:
            y_true: Ground truth labels.
            y_pred: Predicted labels.

        Returns:
            A tensor representing the Jaccard Loss.
        """
        return 1 - LossFunctions.jaccard_index(y_true, y_pred)
    
    def tversky_index(y_true, y_pred, alpha=0.5, beta=0.5, smooth=1):
        """
        Computes the Tversky Index, a generalized form of the Dice Coefficient, 
        that introduces a weighting between false positives and false negatives.
        
        Args:
            y_true: Ground truth labels.
            y_pred: Predicted labels.
            alpha: Weight for false negatives.
            beta: Weight for false positives.
            smooth: A smoothing constant to avoid division by zero.

        Returns:
            A tensor representing the Tversky Index.
        """
        y_true_f = K.flatten(K.cast(y_true, 'float32'))
        y_pred_f = K.flatten(y_pred)
        true_pos = K.sum(y_true_f * y_pred_f)
        false_neg = K.sum(y_true_f * (1 - y_pred_f))
        false_pos = K.sum((1 - y_true_f) * y_pred_f)
        return (true_pos + smooth) / (true_pos + alpha * false_neg + beta * false_pos + smooth)
    
    def tversky_loss(y_true, y_pred, alpha=0.5, beta=0.5):
        """
        Computes the Tversky Loss, derived from the Tversky Index, used to handle 
        class imbalance by adjusting the penalty for false positives and false negatives.
        
        Args:
            y_true: Ground truth labels.
            y_pred: Predicted labels.
            alpha: Weight for false negatives.
            beta: Weight for false positives.

        Returns:
            A tensor representing the Tversky Loss.
        """
        return 1 - LossFunctions.tversky_index(y_true, y_pred, alpha, beta)

### Usage Examples

Para `binary_crossentropy`:

```
model.compile(optimizer='adam', 
              loss=LossFunctions.binary_crossentropy(), 
              metrics=[LossFunctions.binary_accuracy()])
```

Para `dice_loss`:

```
model.compile(optimizer=optimizer, 
              loss=LossFunctions.dice_loss, 
              metrics=[LossFunctions.dice_coefficient])
```


Para `jaccard_loss`:

```
model.compile(optimizer=optimizer, 
              loss=LossFunctions.jaccard_loss, 
              metrics=[LossFunctions.jaccard_index])
```

Para `tversky_loss` (com parâmetros padrão):

```
model.compile(optimizer=optimizer, 
              loss=LossFunctions.tversky_loss, 
              metrics=[LossFunctions.tversky_index])
```

Para `tversky_loss` com parâmetros específicos:

```
model.compile(optimizer=optimizer, 
              loss=lambda y_true, y_pred: LossFunctions.tversky_loss(y_true, y_pred, alpha=0.7, beta=0.3), 
              metrics=[lambda y_true, y_pred: LossFunctions.tversky_index(y_true, y_pred, alpha=0.7, beta=0.3)])
```

## Model compilation

In [ ]:
img_size = (128, 128)
x_model = xception_model(img_size)

x_model.compile(
    optimizer=optimizers.Adam(1e-4), loss="binary_crossentropy",
    metrics=[K.metrics.BinaryAccuracy()],
)

callbks = [
    callbacks.ModelCheckpoint("x_model.keras", save_best_only=True)
]

x_model.summary()

## Train

In [ ]:
start_time = time.time()

history = x_model.fit(
    train_dataset,
    epochs=epochs,
    validation_data=valid_dataset,
    callbacks=callbks,
    verbose=1,
)

end_time = time.time()
training_time = end_time - start_time

# Results

In [ ]:
# show results in this section

## Visualization

In [ ]:
# Implement visualization for predicted outputs

## Model Evaluation

In [ ]:
# Compare metrics and accuracy scores

## Resource Usage Analysis

In [ ]:
# A few interesting points here
# > Training Time
# > Inference Time
# > Memory Usage
# > Batch Size scalability
# > Energy Efficiency (?)